# 맞춤법 수정 with LLM

### colab 세팅

In [1]:
from google.colab import drive
drive.mount('/content/drive')

parent_path = '/content/drive/Othercomputers/My MacBook Pro/colab/'

import sys
sys.path.append(parent_path + 'preprocess')

Mounted at /content/drive


In [ ]:
!python '/content/drive/Othercomputers/My MacBook Pro/colab/settings.py'
%env OLLAMA_MAX_LOADED_MODELS=2
%env OLLAMA_NUM_PARALLEL=2
!nohup ollama serve &

Executing command: ['sudo', 'apt-get', 'install', '-y', 'pciutils']
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libpci3 pci.ids
The following NEW packages will be installed:
  libpci3 pci.ids pciutils
0 upgraded, 3 newly installed, 0 to remove and 29 not upgraded.
Need to get 343 kB of archives.
After this operation, 1,581 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 pci.ids all 0.0~2022.01.22-1 [251 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libpci3 amd64 1:3.7.0-6 [28.9 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 pciutils amd64 1:3.7.0-6 [63.6 kB]
Fetched 343 kB in 1s (262 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 3.)
debc

### 맞춤법 수정 prompt 준비

In [ ]:
import pandas as pd
from utils import load_ollama, run_ollama, parallel_process_n
import random

train_df = pd.read_csv(parent_path + 'data/train_01.csv')
test_df = pd.read_csv(parent_path + 'data/test_01.csv')

In [ ]:
def make_correct_description_prompt(row):
    prompt = f"""당신은 맞춤법 수정 전문가입니다. 제공될 문장의 맞춤법을 수정하되, 형식을 유지하고 추가적인 정보를 생성하지 마세요.
문장: \"{row['description']}\"
수정: """
    return prompt

def model_query_test(df, prompt_func, chain, idx=-1):
    if idx == -1:
        random_idx = random.randint(0, len(df) - 1)
    else:
        random_idx = idx
    print(f"idx: {random_idx}")
    prompt = prompt_func(df.iloc[random_idx])
    print(prompt)

    answer = run_ollama(prompt, chain)
    print(answer)

chain = load_ollama(model="gemma3:4b", temperature=0.0)

train_df['correct_description_prompt'] = train_df.apply(lambda x: make_correct_description_prompt(x), axis=1)
test_df['correct_description_prompt'] = test_df.apply(lambda x: make_correct_description_prompt(x), axis=1)

### 쿼리 테스트

In [ ]:
model_query_test(train_df, make_correct_description_prompt, chain, idx=-1)

idx: 3528
당신은 맞춤법 수정 전문가입니다. 제공될 문장의 맞춤법을 수정하되, 형식을 유지하고 추가적인 정보를 생성하지 마세요.
문장: "조립작업 중 거푸집 관련 넘어짐 사고 (이동식틀비계 발판을 올라가는 도중 온열로 인한 기절로 넘어짐)"
수정: 
조립작업 중 거푸집 관련 넘어짐 사고 (이동식틀비계 발판을 올라가는 도중 온열로 인한 기절로 인해 넘어짐)


### 모델 준비 및 실행

In [ ]:
# 모델 준비
chain1 = load_ollama(model="gemma3:4b", temperature=0.0)
chain2 = load_ollama(model="gemma3:4bcp", temperature=0.0)

In [ ]:
!nohup ollama serve &

nohup: appending output to 'nohup.out'


In [ ]:
no_category_df = test_df[test_df['category_exists']=="N"]

parallel_process_n(no_category_df, 'correct_description_prompt', 'correct_description', chain1, chain2)
test_df[test_df['category_exists']=="N", 'correct_description']

In [ ]:
test_df[test_df['category_exists']=="N", 'correct_description'] = no_category_df['correct_description']

In [ ]:
#--------------------------

In [ ]:
parallel_process_n(test_df, 'correct_description_prompt', 'correct_description', chain1, chain2)
test_df.to_csv(parent_path + 'data/test_02.csv', index=False)

  6%|▌         | 15/241 [00:47<11:51,  3.15s/it]


KeyboardInterrupt: 

In [ ]:
parallel_process_n(train_df, 'correct_description_prompt', 'correct_description', chain1, chain2)
train_df.to_csv(parent_path + 'data/train_02.csv', index=False)